## Opció amb fitxer ttl

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery

# Función para pedir preferencias al usuario
def pedir_preferencias():
    genero = input("¿Qué género estás buscando? ").strip()
    longitud = input("¿Te gustan libros cortos/medios/largos? ").strip().lower()
    autor_favorito = input("¿Cuál es tu autor favorito? ").strip()
    libro_reciente = input("¿Tienes una referencia de algún libro que hayas leído recientemente? ").strip()
    return genero, longitud, autor_favorito, libro_reciente

# Función para crear las consultas SPARQL con base en las preferencias del usuario
def crear_consulta(genero, longitud, autor_favorito, libro_reciente):
    longitud_filter = {
        "corto": "FILTER (?pages <= 200)",
        "medio": "FILTER (?pages > 200 && ?pages <= 400)",
        "largo": "FILTER (?pages > 400)"
    }

    consulta = f"""
    PREFIX ns: <http://example.org/ns#>

    SELECT ?book ?title ?pages ?authorName ?similarToTitle
    WHERE {{
        ?book rdf:type ns:Book .
        ?book ns:title ?title .
      
        # Filtrar por género preferido
        ?book ns:hasGenre ?genreAssignment .
        ?genreAssignment ns:genreType ?genre .
        FILTER (?genre = ns:{genero})  # Reemplaza con el género preferido del usuario
      
        # Filtrar por longitud del libro
        ?book ns:hasPages ?pages .
        {longitud_filter[longitud]}
      
        # Incluir autor favorito si está disponible
        OPTIONAL {{
            ?book ns:writtenBy ?author .
            ?author ns:hasName ?authorName .
            FILTER (?author = ns:{autor_favorito.replace(' ', '_')})  # Reemplaza con el autor favorito del usuario
        }}
      
        # Incluir libros similares a uno que el usuario ha leído recientemente
        OPTIONAL {{
            ?book ns:isSimilarTo ?similarBook .
            ?similarBook ns:title ?similarToTitle .
            FILTER (?similarBook = ns:Book_{libro_reciente.replace(' ', '_')})  # Reemplaza con el libro leído recientemente
        }}
    }}
    ORDER BY ?title
    """
    return consulta

# Función para ejecutar la consulta SPARQL
def ejecutar_consulta(grafo, consulta):
    q = prepareQuery(consulta)
    resultados = grafo.query(q)
    return resultados



In [ ]:
# Cargar el grafo
g = Graph()
g.parse("path/to/your/graph.ttl", format="ttl")  # Ajusta la ruta y el formato según tu archivo de grafo

# Pedir preferencias al usuario
genero, longitud, autor_favorito, libro_reciente = pedir_preferencias()

# Crear consulta SPARQL
consulta = crear_consulta(genero, longitud, autor_favorito, libro_reciente)

# Ejecutar consulta y mostrar resultados
resultados = ejecutar_consulta(g, consulta)

for row in resultados:
    print(f"Book: {row.title}, Pages: {row.pages}, Author: {row.authorName}, Similar to: {row.similarToTitle}")


## Opció amb connexió a GraphDB

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Función para pedir preferencias al usuario
def pedir_preferencias():
    genero = input("¿Qué género estás buscando? ").strip()
    longitud = input("¿Te gustan libros cortos/medios/largos? ").strip().lower()
    autor_favorito = input("¿Cuál es tu autor favorito? ").strip()
    libro_reciente = input("¿Tienes una referencia de algún libro que hayas leído recientemente? ").strip()
    return genero, longitud, autor_favorito, libro_reciente

# Función para crear las consultas SPARQL con base en las preferencias del usuario
def crear_consulta(genero, longitud, autor_favorito, libro_reciente):
    longitud_filter = {
        "corto": "FILTER (?pages <= 200)",
        "medio": "FILTER (?pages > 200 && ?pages <= 400)",
        "largo": "FILTER (?pages > 400)"
    }

    consulta = f"""
    PREFIX ns: <http://example.org/ns#>

    SELECT ?book ?title ?pages ?authorName ?similarToTitle
    WHERE {{
        ?book rdf:type ns:Book .
        ?book ns:title ?title .
      
        # Filtrar por género preferido
        ?book ns:hasGenre ?genreAssignment .
        ?genreAssignment ns:genreType ?genre .
        FILTER (?genre = ns:{genero})  # Reemplaza con el género preferido del usuario
      
        # Filtrar por longitud del libro
        ?book ns:hasPages ?pages .
        {longitud_filter[longitud]}
      
        # Incluir autor favorito si está disponible
        OPTIONAL {{
            ?book ns:writtenBy ?author .
            ?author ns:hasName ?authorName .
            FILTER (?author = ns:{autor_favorito.replace(' ', '_')})  # Reemplaza con el autor favorito del usuario
        }}
      
        # Incluir libros similares a uno que el usuario ha leído recientemente
        OPTIONAL {{
            ?book ns:isSimilarTo ?similarBook .
            ?similarBook ns:title ?similarToTitle .
            FILTER (?similarBook = ns:Book_{libro_reciente.replace(' ', '_')})  # Reemplaza con el libro leído recientemente
        }}
    }}
    ORDER BY ?title
    """
    return consulta

# Función para ejecutar la consulta SPARQL en GraphDB
def ejecutar_consulta(endpoint_url, consulta):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(consulta)
    sparql.setReturnFormat(JSON)
    resultados = sparql.query().convert()
    return resultados


ModuleNotFoundError: No module named 'SPARQLWrapper'

In [ ]:

# URL del endpoint SPARQL de GraphDB
endpoint_url = "http://localhost:7200/repositories/project_books"  # Cambia esto según tu configuración

# Pedir preferencias al usuario
genero, longitud, autor_favorito, libro_reciente = pedir_preferencias()

# Crear consulta SPARQL
consulta = crear_consulta(genero, longitud, autor_favorito, libro_reciente)

# Ejecutar consulta y mostrar resultados
resultados = ejecutar_consulta(endpoint_url, consulta)

for resultado in resultados["results"]["bindings"]:
    book = resultado["book"]["value"]
    title = resultado["title"]["value"]
    pages = resultado["pages"]["value"] if "pages" in resultado else "N/A"
    author_name = resultado["authorName"]["value"] if "authorName" in resultado else "N/A"
    similar_to_title = resultado["similarToTitle"]["value"] if "similarToTitle" in resultado else "N/A"
    print(f"Book: {title}, Pages: {pages}, Author: {author_name}, Similar to: {similar_to_title}")
